In [ ]:
!pip install streamlit

In [10]:
import os

os.chdir('/content/drive/MyDrive/Omdena/Osun Nigeria/App')

!pwd

In [31]:
from google.colab import files
uploaded = files.upload()

Saving Corn_Common_Rust (1).jpg to Corn_Common_Rust (1).jpg


In [100]:
%%writefile app.py

import streamlit as st
from apps import home, crop_disease_detection
from PIL import Image

with st.container():
    proj_title_col, logo_col = st.columns([6,1])

    with proj_title_col:
      st.subheader('Improving Food Security in Nigeria')

    with logo_col:
      logo = Image.open('/content/drive/MyDrive/Omdena/Osun Nigeria/App/Corn_Common_Rust (1).jpg')
      logo = logo.resize((100,100))
      st.image(logo)

PAGES = {
    "Home": home,
    "Crop Disease Detection": crop_disease_detection
}

selection = st.sidebar.radio("Menu", list(PAGES.keys()))
page = PAGES[selection]

page.app()

Overwriting app.py


In [75]:
%%writefile apps/home.py

import streamlit as st

def app():
  st.subheader('Problem Statement')
  st.write("According to the Food and Agriculture Organisation of the United Nations, 2018, \
  approximately 88 % of the farmers in Nigeria engage in \
  agricultural production at a subsistence level, and they lack sustainable \
  farming knowledge and practices. Also, Nigeria is endowed with \
  different climatic conditions and soil quality which leads to lackluster \
  crop production. ")

  st.subheader('Objectives of the Project')
  st.write('This project is aimed at helping farmers to boost their \
  farm produce and plan their farming system.')

Overwriting apps/home.py


In [97]:
%%writefile apps/crop_disease_detection.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image

TARGET_SIZE = (100, 100)


@st.cache(allow_output_mutation=True)

def load_model():
  model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_maize.h5')
  return model

def preprocess_img(image):
  img = image.resize(TARGET_SIZE)
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = preprocess_input(img)
  return img

def predict(image, model):
  predictions = model.predict(image)
  return predictions

def get_class(idx):

  CLASSES = {
      0: 'Blight',
      1: 'Common_Rust',
      2: 'Gray_Leaf_Spot',
      3: 'Healthy'
  }

  return CLASSES[idx]

def app():
  
  new_image = ''
  has_pred = False
  has_replaced_image = False
  pred = ''

  #Load the model
  model = load_model()

  with st.container():
      upload_col, predict_col = st.columns([6,1])
      with upload_col:
        uploaded_file = st.file_uploader('Choose a file',type=["jpg"])
        if uploaded_file is not None:
          img = Image.open(uploaded_file)
          img_display = img.resize((500, 500))
          st.image(img_display)

          #Preprocess image
          img = preprocess_img(img)
          new_image = img
          has_replaced_image = True
          
          
      with predict_col:
          if st.button('Predict'):
            if has_replaced_image:
              predictions = predict(new_image, model)
              pred = np.argmax(predictions)
              has_pred = True
            else:
              st.error('Please upload an image')
              
  with st.container():
    if has_pred:
      with st.expander("Read disease description"):
        disease_class = get_class(pred)
        if disease_class == 'Blight':
          st.write('Write the blight disease description here.')
        elif disease_class == 'Common_Rust':
          st.write('Write the common rust disease description here.')
        elif disease_class == 'Gray_Leaf_Spot':
          st.write('Write the gray leaf spot disease description here.')
        elif disease_class == 'Healthy':
          st.write('Write healthy description here.')

Overwriting apps/crop_disease_detection.py


In [101]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 3.109s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.229.32.166:8501

your url is: https://warm-vampirebat-10.loca.lt
2021-10-22 08:41:23.249326: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-22 08:41:23.249407: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a62acd758be1): /proc/driver/nvidia/version does not exist
2021-10-22 08:41:35.555119: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
  Stopping...
^C
